In [46]:
import xlsxwriter
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import xlwt

i,j,k,l = 0,0,0,0
row = 0
col = 0
N = 3 #number of files to read +1
Max_G = 1700 #maximum index of fitted Raman peak
Min_G = 1500 #minimum index of fitted Raman peak
Max_2D = 2900 #maximum index of fitted Raman peak
Min_2D = 2500 #minimum index of fitted Raman peak
conditions_2D = [0.5, 100, 100000, 20, 2650] #the initial conditions for the fit
conditions_G = [0.5, 100, 100000, 20, 1599]
measurement_no = list(np.linspace(0,N,1))
name = "mapa_"

xbook = xlsxwriter.Workbook('Output_G_2D.xlsx')
def output(filename, sheet, list1, list2, x, y, z):
    book = xlwt.Workbook()
    sh = book.add_sheet(sheet)
    
xsheet_2D = xbook.add_worksheet('2D_peak')
xsheet_G = xbook.add_worksheet('G_peak')
    
#Lorentian or double-lorentian function 
def double_Lorentz(x, a, b, A, w, x_0, A1, w1, x_01):
    return a*x+b+(2*A/np.pi)*(w/(4*(x-x_0)**2 + w**2))+(2*A1/np.pi)*(w1/(4*(x-x_01)**2 + w1**2))

def Lorentz(x, a, b, A, w, x_0):
    return a*x+b+(2*A/np.pi)*(w/(4*(x-x_0)**2 + w**2))

for m in range(0,N):
    data = np.loadtxt(name + str(m) +'.txt')
    print("reading file no " + str(m)) #debug
    x = data[:,0]
    y = data[:,1]
    wave = data[:,2]
    intensity = data[:,3]

    list_waves, list_intensities, list_x, list_y = [], [], [], []
    curr_series_beggining = 0
    wave_diff_bool = (wave[1:] - wave[:-1]) > 0

    beggining_indices = np.where(wave_diff_bool)
    beggining_indices = np.append(0, beggining_indices) #zero index
    beggining_indices = np.append(beggining_indices, len(wave)) #1st index

    for i in range(len(beggining_indices[:-1])): #extracting single spectrums from a mapping measurement file
        beginning = beggining_indices[i]
        ending = beggining_indices[i+1]
        if i==0: 
            current_wave = wave[beginning : ending+1]
            current_intensity = intensity[beginning : ending+1]
            current_x = x[beginning : ending+1]
#             current_y = y[beginning : ending+1]
        else:
            current_wave = wave[beginning+1 : ending+1]
            current_intensity = intensity[beginning+1 : ending+1]
            current_x = x[beginning+1 : ending+1]
#             current_y = y[beginning+1 : ending+1]
        
        list_waves.append(current_wave)
        list_intensities.append(current_intensity)
        list_x.append(current_x)
#         list_y.append(current_y)
    
    x_G, w_G, A_G, x_2D, w_2D, A_2D = [], [], [], [], [], []
    
    n = 0
    index_G_end=0 
    index_G_begin=0
    index_2D_end=0 
    index_2D_begin=0
    
# #2D mode
    n = 0
    for i in range(len(list_waves)):
        while list_waves[i][n] > Max_2D: #maximum frequency of a peak to fit
            n = n+1
            index_2D_begin=n
        while list_waves[i][n] > Min_2D: #minimum frequency of a peak to fit
            n = n+1
            index_2D_end=n
        n=1
    
    for i in range(len(list_waves)):
            intens_plot_2D =list_intensities[i][index_2D_begin:index_2D_end]
            wave_plot_2D =list_waves[i][index_2D_begin:index_2D_end]
            parameters, covariances = curve_fit(Lorentz, wave_plot_2D, intens_plot_2D, conditions_2D)
            a, b, A, w, x_0 = parameters
            x_fit = np.linspace(wave_plot_2D.min(), wave_plot_2D.max(), 1000)
            y_fit = Lorentz(x_fit, a, b, A, w, x_0)
            x_2D.append(x_0) #x_0 - position of the fitted peak
            w_2D.append(w) #full width at half maximum of the fitted peak
            A_2D.append(A) #intensity of the fitted peak
        
# G mode            
    n = 0
    for i in range(len(list_waves)):
        while list_waves[i][n] > Max_G: #maximum frequency of a peak to fit
            n = n+1
            index_G_begin=n
        while list_waves[i][n] > Min_G: #minimum frequency of a peak to fit
            n = n+1
            index_G_end=n
        n=1
    
    for i in range(len(list_waves)):
            intens_plot_G =list_intensities[i][index_G_begin:index_G_end]
            wave_plot_G =list_waves[i][index_G_begin:index_G_end]
            parameters, covariances = curve_fit(Lorentz, wave_plot_G, intens_plot_G, conditions_G)
            a, b, A, w, x_0 = parameters
            x_fit = np.linspace(wave_plot_G.min(), wave_plot_G.max(), 1000)
            y_fit = Lorentz(x_fit, a, b, A, w, x_0)
            x_G.append(x_0) #x_0 - position of the fitted peak
            w_G.append(w) #full width at half maximum of the fitted peak
            A_G.append(A) #intensity of the fitted peak

#write to spreadsheet

    for idx, number in enumerate(x_G):
        xsheet_G.write(idx, j, x_G[idx])
        j+=1
        xsheet_G.write(idx, j, w_G[idx])
        j+=1
        xsheet_G.write(idx, j, A_G[idx])
        j=0
        
    for idx, number in enumerate(x_2D):
        xsheet_2D.write(idx, j, x_2D[idx])
        j+=1
        xsheet_2D.write(idx, j, w_2D[idx])
        j+=1
        xsheet_2D.write(idx, j, A_2D[idx])
        j=0
        
xbook.close()
print("Done!")

reading file no 0
reading file no 1
reading file no 2
Done!


array([ 2897.203125,  2895.703125,  2894.201172, ...,  1221.435547,
        1219.574219,  1217.708984])